## Rfam pull

In [ ]:
import os

def name(idx):
    return "RF{}.fa".format(str(idx).zfill(5))

def link(idx):
    return "https://ftp.ebi.ac.uk/pub/databases/Rfam/CURRENT/fasta_files/{}.gz".format(name(idx))

for i in range(4300, 4301):
    os.system(f"wget -P /Users/yvesmartin/data/rfam {link(i)}")
    os.system(f"gunzip /Users/yvesmartin/data/rfam/{name(i)}.gz")
    os.system(f"rm /Users/yvesmartin/data/rfam/{name(i)}.gz")

In [ ]:
for f in os.listdir("/Users/yvesmartin/data/rfam"):
    if f.endswith(".gz.1"):
        os.system(f"rm /Users/yvesmartin/data/rfam/{f}")

## Make it a dataset

In [10]:
import os

def fasta_parser(f):
    while line:=f.readline():
        if line.startswith(">"):
            ref = line[1:].split()[0]
            desc = ' '.join(line.split()[1:]).strip()
        else:
            seq = line.strip()
            yield ref, desc, seq
            
fams, refs, descs, seqs = [], [], [], []
refs_count = {}
for f in os.listdir("/Users/yvesmartin/data/rfam/fastas"):
    if f.endswith(".fa"):
        with open("/Users/yvesmartin/data/rfam/fastas/{}".format(f), "r") as f:
            for ref, desc, seq in fasta_parser(f):
                fams.append(os.path.basename(f.name).replace(".fa", ""))
                if ref in refs_count:
                    refs.append("{}_{}".format(ref, refs_count[ref]))
                else:
                    refs.append(ref)
                descs.append(desc)
                seqs.append(seq)
                
                refs_count[ref] = refs_count.get(ref, 0) + 1
                
import pandas as pd
df = pd.DataFrame({"family": fams, "reference": refs, "ref_desc": descs, "sequence": seqs})#.set_index("reference")

## Add family and clan info

### Clan

In [12]:
df_clan = pd.read_csv('clan.txt', sep='\t', header=None, usecols=[0,1,3], names=['clan', 'clan_name', 'full_clan_name'])
df_clan

,clan,clan_name,full_clan_name
0,CL00001,tRNA,tRNA clan
1,CL00002,RNaseP,RNaseP clan
2,CL00003,SRP,SRP clan
3,CL00004,Telomerase,Telomerase clan
4,CL00005,U1,U1 clan
...,...,...,...
141,CL00145,MIR1520,MIR1520 microRNA precursor clan
142,CL00146,mir-8791,mir-8791 microRNA precursor clan
143,CL00147,mir-574,mir-574 microRNA precursor clan
144,CL00148,let-7,let-7 microRNA precursor clan


### Family

In [13]:
df_fam = pd.read_csv('fam.txt', sep='\t', header=None, usecols=[0,1,3], names=['family', 'family_name', 'full_family_name'], encoding='latin-1')
df_fam

,family,family_name,full_family_name
0,RF00001,5S_rRNA,5S ribosomal RNA
1,RF00002,5_8S_rRNA,5.8S ribosomal RNA
2,RF00003,U1,U1 spliceosomal RNA
3,RF00004,U2,U2 spliceosomal RNA
4,RF00005,tRNA,tRNA
...,...,...,...
4165,RF04296,mir-485,mir-485 microRNA precursor family
4166,RF04297,mir-35_2,mir-35_2 microRNA precursor family
4167,RF04298,mir-36_2,mir-36_2 microRNA precursor family
4168,RF04299,MIR814,MIR814 microRNA precursor family


### Family to clan mapping

In [14]:
df_clan2fam = pd.read_csv('clan2fam.txt', sep='\t', header=None, usecols=[0,1], names=['clan', 'family'])
df_clan2fam

,clan,family
0,CL00001,RF00005
1,CL00001,RF00023
2,CL00001,RF01849
3,CL00001,RF01850
4,CL00001,RF01851
...,...,...
450,CL00149,RF04292
451,CL00149,RF04293
452,CL00149,RF04294
453,CL00149,RF04295


### Add family / clan info to the df

In [16]:
df = pd.merge(df, df_fam, on='family', how='left')
df = pd.merge(df, df_clan2fam, on='family', how='left')
df = pd.merge(df, df_clan, on='clan', how='left')

In [17]:
df['family'].nunique()

450

In [7]:
df.to_csv('data/rfam.csv', index=False)
df.set_index('reference').to_json('data/rfam.json', orient='index', indent=2)

### Upload data to HF

In [9]:
from rouskinhf import upload_dataset
import envbash
envbash.load.load_envbash('../.env')

upload_dataset(
    'data/rfam.json',
)

rfam.json:   0%|          | 0.00/1.03G [00:00<?, ?B/s]